In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



In [20]:
import torch
from torch import nn
import torchlibrosa
import numpy as np
import onnx_coreml


class WaveToSpectrogram(nn.Module):

    def __init__( self, n_fft, hop_length, center=False ):
        super(WaveToSpectrogram, self).__init__()

        self.spec_extractor = torchlibrosa.stft.Spectrogram(
            n_fft=n_fft,
            hop_length=hop_length,
            center=center,
        )

    def forward( self, x ):
        return self.spec_extractor( x )
    
    def convert_to_onnx( self, filename_onnx, sample_input ):
        input_names = [ 'input.1' ]
        output_names = ['Spectrogram']

        torch.onnx.export(
            self,
            torch.from_numpy( sample_input ),
            filename_onnx,
            input_names=input_names,
            output_names=output_names,                     
        )

In [5]:
# load in waveform
import soundfile



In [19]:
model = WaveToSpectrogram( n_fft=1024, hop_length=512 )
torch.save(model, 'Hello.pt')


/Users/hugo/opt/anaconda3/envs/audio/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)


## Convert to Onnx

In [12]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")
device=get_default_device()

In [ ]:
import soundfile
import numpy as np
import os
model_path = 'history/Cnn10_lr_0.005_ep_100_bs_32_valoss_1.5_acc_0.94_recall_0.89_precision_0.89'
waveform, samplerate = soundfile.read( '../urbansound8k/audio/fold1/101415-3-0-2.wav' )
sample_input = waveform[:192000].astype( dtype=np.float32).reshape((1, 192000))

def export_torch_onnx(model, sample_input, filename_onnx):
    torch.onnx.export(
            model,
            torch.from_numpy( sample_input).to(device),
            filename_onnx,
            verbose = True,
            )
model = torch.load(os.path.join(model_path, 'model.pt'))
export_torch_onnx(model, sample_input, os.path.join(model_path, 'model.onnx'))


## Convert to Coreml

In [4]:
import onnx_coreml
import coremltools

mlmodel = onnx_coreml.convert(
    model = os.path.join(model_path, 'model.onnx'),
    predicted_feature_name = [],
    minimum_ios_deployment_target='13',    
)
mlmodel.save(os.path.join(model_path, 'model.mlmodel'))

ModuleNotFoundError: No module named 'coremltools.libmilstoragepython'